# Week 6 — LLM-as-Judge: Automated Evaluation with Language Models
### BenchRight LLM Evaluation Master Program (18 Weeks)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. Understand the LLM-as-Judge evaluation paradigm
2. Use the `LLMJudge` class from `src/benchmark_engine`
3. Evaluate model answers using an LLM as the judge
4. Compare LLM-as-Judge with exact match metrics
5. Analyze when different metrics disagree

---

## 🧠 What is LLM-as-Judge?

### The Challenge

Traditional metrics like exact match are too rigid:

| Question | Reference | Model Answer | Exact Match | Correct? |
|----------|-----------|--------------|-------------|----------|
| Capital of France? | Paris | Paris | 1.0 ✓ | Yes |
| Capital of France? | Paris | The capital is Paris | 0.0 ✗ | Yes! |
| Capital of France? | Paris | Paris, France | 0.0 ✗ | Yes! |

### The Solution

Use a powerful LLM (like GPT-4) to evaluate answers like a human would:
- Understands semantics, not just string matching
- Can handle paraphrases and elaborations
- Provides rationale for the score

---

## 🛠️ Step 1: Setup & Dependencies

In [ ]:
# Standard library imports
import sys
import json
from typing import Any, Dict, List

# Add src to path if running in Colab
sys.path.insert(0, '.')

print("✅ Setup complete!")

---

## 📦 Step 2: Import the LLMJudge Class

In [ ]:
# Import the LLMJudge class
from src.benchmark_engine.llm_judge import LLMJudge, JUDGE_SYSTEM_PROMPT

print("✅ LLMJudge imported successfully!")
print(f"\n📋 System Prompt Preview (first 200 chars):")
print(JUDGE_SYSTEM_PROMPT[:200] + "...")

---

## 🤖 Step 3: Create an LLM Client

### Option A: Using OpenAI API (requires API key)

```python
# Uncomment and run if you have an OpenAI API key
# !pip install openai
# from openai import OpenAI
# client = OpenAI()  # Uses OPENAI_API_KEY env variable
```

### Option B: Using a Mock Client (for demonstration)

In [ ]:
class MockChatCompletions:
    """Mock chat completions that returns predefined responses."""
    
    def __init__(self):
        # Define expected responses for different scenarios
        self.responses = {
            # Correct answers
            "paris": '{"score": 1.0, "rationale": "The answer correctly identifies Paris as the capital of France, matching the reference answer perfectly."}',
            "the capital is paris": '{"score": 0.95, "rationale": "The answer correctly identifies Paris as the capital. The additional phrasing provides context without reducing accuracy."}',
            "4": '{"score": 1.0, "rationale": "The answer correctly states that 2+2 equals 4, matching the reference exactly."}',
            "the sum is 4": '{"score": 0.95, "rationale": "The answer correctly provides the sum of 2+2 as 4. The phrasing is clear and helpful."}',
            "jupiter": '{"score": 1.0, "rationale": "The answer correctly identifies Jupiter as the largest planet in our solar system."}',
            
            # Incorrect answers
            "mars": '{"score": 0.0, "rationale": "The answer is incorrect. Mars is not the largest planet; Jupiter is the largest planet in our solar system."}',
            "london": '{"score": 0.0, "rationale": "The answer is incorrect. London is not the capital of France; Paris is the capital."}',
            "5": '{"score": 0.0, "rationale": "The answer is mathematically incorrect. 2+2 equals 4, not 5."}',
            
            # Partial credit
            "i think it might be paris": '{"score": 0.7, "rationale": "The answer correctly identifies Paris but expresses unnecessary uncertainty, reducing confidence."}',
            "h2o": '{"score": 1.0, "rationale": "The answer correctly identifies H2O as the chemical formula for water."}',
            "water is h2o": '{"score": 0.95, "rationale": "The answer correctly identifies H2O as the formula for water with helpful context."}',
        }
        
        # Default response for unknown inputs
        self.default_response = '{"score": 0.5, "rationale": "The answer partially addresses the question but could be more precise."}'
    
    def create(self, model: str, messages: List[Dict], temperature: float = 0.0):
        """Simulate a chat completion API call."""
        # Extract the user message to determine the answer being evaluated
        user_msg = messages[-1]["content"].lower()
        
        # Find the answer in the user message
        response_text = self.default_response
        for key, value in self.responses.items():
            if f"answer: {key}" in user_msg:
                response_text = value
                break
        
        # Create mock response object
        class MockMessage:
            content = response_text
        
        class MockChoice:
            message = MockMessage()
        
        class MockResponse:
            choices = [MockChoice()]
        
        return MockResponse()


class MockChat:
    """Mock chat interface."""
    completions = MockChatCompletions()


class MockOpenAIClient:
    """Mock OpenAI client for demonstration without API key."""
    chat = MockChat()


# Create the mock client
mock_client = MockOpenAIClient()
print("✅ Mock OpenAI client created!")
print("   (Replace with real OpenAI client for production use)")

---

## 🏛️ Step 4: Initialize the LLMJudge

In [ ]:
# Create the LLMJudge with our client
judge = LLMJudge(
    client=mock_client,
    model="gpt-4o-mini",  # Model name (mock ignores this)
)

print("✅ LLMJudge initialized!")
print(f"   Model: {judge.model}")

---

## 🧪 Step 5: Test Single Evaluation

In [ ]:
# Test a single evaluation
result = judge.score_answer(
    question="What is the capital of France?",
    answer="Paris",
    reference="Paris"
)

print("📝 Single Evaluation Test:")
print("=" * 60)
print(f"Question:  What is the capital of France?")
print(f"Answer:    Paris")
print(f"Reference: Paris")
print(f"\n📊 Result:")
print(f"   Score:     {result['score']:.2f}")
print(f"   Rationale: {result['rationale']}")

---

## 📊 Step 6: Evaluate Multiple Test Cases

In [ ]:
# Define test cases with varying correctness levels
test_cases = [
    # Exact matches (should score 1.0)
    {
        "question": "What is the capital of France?",
        "answer": "Paris",
        "reference": "Paris",
        "expected_high": True
    },
    {
        "question": "What is 2+2?",
        "answer": "4",
        "reference": "4",
        "expected_high": True
    },
    # Correct with elaboration (should score high)
    {
        "question": "What is the capital of France?",
        "answer": "The capital is Paris",
        "reference": "Paris",
        "expected_high": True
    },
    # Incorrect answers (should score low)
    {
        "question": "What is the largest planet?",
        "answer": "Mars",
        "reference": "Jupiter",
        "expected_high": False
    },
    {
        "question": "What is the capital of France?",
        "answer": "London",
        "reference": "Paris",
        "expected_high": False
    },
    # Uncertain but correct (partial credit)
    {
        "question": "What is the capital of France?",
        "answer": "I think it might be Paris",
        "reference": "Paris",
        "expected_high": True  # Partially
    },
]

print("📊 Evaluating Multiple Test Cases:")
print("=" * 80)

results = []
for i, tc in enumerate(test_cases, 1):
    result = judge.score_answer(
        question=tc["question"],
        answer=tc["answer"],
        reference=tc["reference"]
    )
    
    # Store result
    results.append({
        "question": tc["question"],
        "answer": tc["answer"],
        "reference": tc["reference"],
        "score": result["score"],
        "rationale": result["rationale"],
        "expected_high": tc["expected_high"]
    })
    
    # Display
    status = "✓" if (result["score"] >= 0.7) == tc["expected_high"] else "✗"
    print(f"\n[{status}] Test {i}:")
    print(f"    Q: {tc['question']}")
    print(f"    A: {tc['answer']}")
    print(f"    R: {tc['reference']}")
    print(f"    Score: {result['score']:.2f}")
    print(f"    Rationale: {result['rationale'][:80]}...")

---

## 📈 Step 7: Compare with Exact Match Metric

In [ ]:
# Import exact match metric from Week 5
from src.benchmark_engine.engine import exact_match_metric

print("📊 Comparison: LLM-as-Judge vs Exact Match")
print("=" * 80)
print(f"{'Answer':<30} {'Reference':<15} {'Exact':<8} {'LLM':<8} {'Diff':<8}")
print("-" * 80)

for r in results:
    exact_score = exact_match_metric(r["answer"], r["reference"])
    llm_score = r["score"]
    diff = llm_score - exact_score
    
    # Highlight disagreements
    marker = "⚠️" if abs(diff) > 0.5 else "  "
    
    print(f"{r['answer'][:28]:<30} {r['reference']:<15} {exact_score:<8.2f} {llm_score:<8.2f} {diff:+.2f} {marker}")

---

## 🔍 Step 8: Analyze Disagreements

In [ ]:
print("\n🔍 Analysis of Metric Disagreements:")
print("=" * 80)

disagreements = []
for r in results:
    exact_score = exact_match_metric(r["answer"], r["reference"])
    llm_score = r["score"]
    
    if abs(llm_score - exact_score) > 0.3:
        disagreements.append({
            "answer": r["answer"],
            "reference": r["reference"],
            "exact": exact_score,
            "llm": llm_score,
            "rationale": r["rationale"]
        })

if disagreements:
    print(f"\nFound {len(disagreements)} significant disagreements:\n")
    for i, d in enumerate(disagreements, 1):
        print(f"  {i}. Answer: \"{d['answer']}\"")
        print(f"     Reference: \"{d['reference']}\"")
        print(f"     Exact Match: {d['exact']:.2f}, LLM Judge: {d['llm']:.2f}")
        print(f"     Rationale: {d['rationale'][:100]}...")
        print()
else:
    print("No significant disagreements found.")

print("\n💡 Key Insight:")
print("   LLM-as-Judge can recognize semantically correct answers that")
print("   exact match would fail. This is especially valuable when:")
print("   - Answers are paraphrased but correct")
print("   - Answers include helpful context")
print("   - The reference is a short form of a longer valid answer")

---

## 🔧 Step 9: Using LLMJudge with the Benchmark Engine

In [ ]:
from src.benchmark_engine import run_benchmark

# Create a simple mock model
def mock_model(prompt: str) -> str:
    """A mock model that returns somewhat elaborated answers."""
    answers = {
        "capital of france": "The capital is Paris",
        "2+2": "The sum is 4",
        "largest planet": "Jupiter",
        "formula for water": "Water is H2O",
    }
    prompt_lower = prompt.lower()
    for key, answer in answers.items():
        if key in prompt_lower:
            return answer
    return "I don't know"


# Dataset
dataset = [
    ("What is the capital of France?", "Paris"),
    ("What is 2+2?", "4"),
    ("What is the largest planet?", "Jupiter"),
    ("What is the chemical formula for water?", "H2O"),
]

# Run with exact match
print("📊 Benchmark Results: Exact Match vs LLM-as-Judge")
print("=" * 60)

# Exact match benchmark
exact_results = run_benchmark(
    model_fn=mock_model,
    dataset=iter(dataset),
    metric_fn=exact_match_metric
)

print(f"\n📐 Exact Match Results:")
print(f"   Mean Score: {exact_results['mean_score']:.2%}")

# LLM-as-Judge evaluation (manual loop since metric needs question context)
llm_scores = []
print(f"\n🤖 LLM-as-Judge Results:")
for question, reference in dataset:
    answer = mock_model(question)
    result = judge.score_answer(question, answer, reference)
    llm_scores.append(result["score"])
    print(f"   {question[:30]}... -> {result['score']:.2f}")

llm_mean = sum(llm_scores) / len(llm_scores)
print(f"\n   Mean Score: {llm_mean:.2%}")

print(f"\n📈 Comparison:")
print(f"   Exact Match: {exact_results['mean_score']:.2%}")
print(f"   LLM-as-Judge: {llm_mean:.2%}")
print(f"   Difference: {(llm_mean - exact_results['mean_score'])*100:+.1f}%")

---

## 📋 Step 10: View the System Prompt

In [ ]:
print("📋 Full System Prompt Used by LLMJudge:")
print("=" * 80)
print(JUDGE_SYSTEM_PROMPT)
print("=" * 80)

---

## 🎓 Mini-Project: Create Your Own Evaluation Pipeline

### Task

Create a dataset of 10 QA pairs and compare exact match with LLM-as-Judge.

### Template

In [ ]:
# Your custom evaluation dataset
my_dataset = [
    # Add 10 QA pairs
    # ("Question 1?", "Reference 1"),
    # ("Question 2?", "Reference 2"),
    # ...
]

# Your model function (or use mock_model)
def my_model(prompt: str) -> str:
    """Your model implementation here."""
    pass

# Evaluate with both metrics
# for question, reference in my_dataset:
#     answer = my_model(question)
#     exact = exact_match_metric(answer, reference)
#     llm_result = judge.score_answer(question, answer, reference)
#     print(f"Q: {question}")
#     print(f"A: {answer}")
#     print(f"Exact: {exact:.2f}, LLM: {llm_result['score']:.2f}")
#     print()

---

## 🤔 Paul-Elder Critical Thinking Questions

Reflect on these questions:

### Question 1: EVIDENCE
**If LLM-as-Judge gives a high score to an incorrect answer, what might explain this?**
*Consider: Model hallucinations, prompt engineering, domain expertise.*

### Question 2: ASSUMPTIONS
**What assumptions are we making when using GPT-4 to judge GPT-3.5 outputs?**
*Consider: Model biases, shared training data, capability differences.*

### Question 3: IMPLICATIONS
**If an organization uses LLM-as-Judge as the only evaluation metric, what could go wrong?**
*Consider: Adversarial inputs, edge cases, accountability.*

---

## ✅ Knowledge Mastery Checklist

Before moving to Week 7, ensure you can check all boxes:

- [ ] I understand what LLM-as-Judge is and when to use it
- [ ] I can use the `LLMJudge` class to evaluate answers
- [ ] I understand the three evaluation criteria (correctness, coherence, helpfulness)
- [ ] I can compare LLM-as-Judge with exact match metrics
- [ ] I understand potential failure modes of LLM-as-Judge
- [ ] I know how to customize the system prompt for different tasks

---

**Week 6 Complete!** 🎉

**Next:** *Week 7 — Semantic Similarity Metrics (Embeddings, Cosine Similarity)*